In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 29.5 MB/s eta 0:00:00


In [ ]:
import torch
torch.__version__

'2.5.1+cu121'

In [ ]:
import transformers
transformers.__version__

'4.46.2'

In [ ]:
!nvidia-smi

Tue Nov 26 05:06:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

tokenizer.model_max_length

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


512

In [ ]:
!git clone https://github.com/declare-lab/MELD.git

fatal: destination path 'MELD' already exists and is not an empty directory.


In [ ]:
import glob
data_path = '/content/MELD/data/MELD/*.csv'
data_path_list = glob.glob(data_path)
print(data_path_list)

['/content/MELD/data/MELD/test_sent_emo.csv', '/content/MELD/data/MELD/train_sent_emo.csv', '/content/MELD/data/MELD/dev_sent_emo.csv']


In [ ]:
!head -5 '/content/MELD/data/MELD/dev_sent_emo.csv'

Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
1,"Oh my God, he’s lost it. He’s totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
3,"Or! Or, we could go to the bank, close our accounts and cut them off at the source.",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
4,You’re a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"


In [ ]:
import csv
for data_path in data_path_list:
  with open(data_path, 'r') as f:
    rdr = csv.reader(f)

    for line in rdr:
      print(line)
      break
    break

['Sr No.', 'Utterance', 'Speaker', 'Emotion', 'Sentiment', 'Dialogue_ID', 'Utterance_ID', 'Season', 'Episode', 'StartTime', 'EndTime']


In [ ]:
from torch.utils.data import Dataset
from transformers import RobertaTokenizer
import csv
import torch

def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

class MeldDataset(Dataset):
    def __init__(self, data_path, context_window=None, max_length=512):
        with open(data_path, 'r') as f:
            rdr = csv.reader(f)
            emoSet = set()
            self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

            self.session_dataset = []
            self.context_window = context_window
            self.max_length = max_length  # max_length 설정 추가
            session = []
            speaker_set = []

            pre_sess = 'start'
            for i, line in enumerate(rdr):
                if i == 0:
                    header = line
                    utt_idx = header.index('Utterance')
                    speaker_idx = header.index('Speaker')
                    emo_idx = header.index('Emotion')
                    sess_idx = header.index('Dialogue_ID')
                else:
                    utt = line[utt_idx]
                    speaker = line[speaker_idx]
                    # speaker indexing
                    if speaker in speaker_set:
                        uniq_speaker = speaker_set.index(speaker)
                    else:
                        speaker_set.append(speaker)
                        uniq_speaker = speaker_set.index(speaker)
                    emotion = line[emo_idx]
                    sess = line[sess_idx]

                    if pre_sess == 'start' or sess == pre_sess:
                        session.append([uniq_speaker, utt, emotion])
                    else:
                        # 이전 데이터 저장
                        self.session_dataset += split(session)
                        session = [[uniq_speaker, utt, emotion]]
                        speaker_set = []
                        emoSet.add(emotion)
                    pre_sess = sess

            self.session_dataset += split(session)

            # self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
            self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

    def __len__(self):
        return len(self.session_dataset)

    def __getitem__(self, idx):
        return self.session_dataset[idx]

    def padding(self, batch_input_token):
        # tokenizer를 통해 이미 max_length로 패딩된 상태
        return batch_input_token['input_ids'], batch_input_token['attention_mask']

    def collate_fn(self, sessions):
        batch_input, batch_labels = [], []
        batch_PM_input = []

        for session in sessions:
            input_str = self.tokenizer.cls_token

            current_speaker, current_utt, current_emotion = session[-1]
            PM_input = []

            # context window : 입력 길이 조절 여부
            if self.context_window is not None:
                session = session[-self.context_window:]

            for i, line in enumerate(session):
                speaker, utt, emotion = line

                # CoM 입력
                input_str += " " + utt + self.tokenizer.sep_token

                # 마지막 발화의 감정을 예측하기 위해 마지막 화자의 발화만 추가함
                if i < len(session)-1 and current_speaker == speaker:
                    PM_input.append(self.tokenizer.encode(utt, add_special_tokens=True, return_tensors='pt'))

            batch_input.append(input_str)
            batch_labels.append(self.emoList.index(emotion))
            batch_PM_input.append(PM_input)

        # max_length, padding, truncation 옵션을 추가하여 토크나이징
        batch_input_token = self.tokenizer(
            batch_input,
            max_length=self.max_length,  # max_length 지정
            padding='max_length',  # max_length로 패딩
            truncation=True,  # 초과하는 부분 잘라내기
            return_tensors='pt'
        )

        batch_padding_token, batch_padding_attention_mask = self.padding(batch_input_token)

        return batch_padding_token, batch_padding_attention_mask, batch_PM_input, torch.tensor(batch_labels)


In [ ]:
""" 배치 결과 확인 """
from torch.utils.data import DataLoader
dev_dataset = MeldDataset('/content/MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=0, collate_fn=dev_dataset.collate_fn)

for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

torch.Size([3, 512])
batch_padding_token tensor([[   0,    0, 1793,  ...,    1,    1,    1],
        [   0,    0, 1793,  ...,    1,    1,    1],
        [   0,    0, 1793,  ...,    1,    1,    1]])
batch_padding_attention_mask tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
batch_PM_input [[], [], [tensor([[    0,  1185,    17,    27,   241,    10, 16333,   328,     2]])]]
batch_label tensor([3, 5, 4])


In [ ]:
!touch dataset.py

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import lightning.pytorch as pl
from transformers import RobertaModel
from torch.utils.data import DataLoader, Dataset
# from dataset import MeldDataset

class MeldDataLoader(pl.LightningDataModule):
  def __init__(self, train_dir: str, valid_dir: str, batch_size: int = 32, context_window=None):
    super().__init__()
    self.train_dir = train_dir
    self.valid_dir = valid_dir
    self.batch_size = batch_size
    self.context_window = context_window

  def setup(self, stage=None):
    self.train_dataset = MeldDataset(self.train_dir, context_window=self.context_window)
    self.val_dataset = MeldDataset(self.valid_dir, context_window=self.context_window)

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, collate_fn=self.train_dataset.collate_fn)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4, collate_fn=self.val_dataset.collate_fn)


In [ ]:
!touch dataloader.py

In [ ]:
from transformers import RobertaModel
import torch
import torch.nn as nn

class ERC_model(nn.Module):
    def __init__(self, clsNum):
        super(ERC_model, self).__init__()
        self.com_model = RobertaModel.from_pretrained('roberta-base')
        self.pm_model = RobertaModel.from_pretrained('roberta-base')

        # GRU 세팅
        self.hiddenDim = self.com_model.config.hidden_size
        self.speakerGRU = nn.GRU(self.hiddenDim, self.hiddenDim, num_layers=2, dropout=0.3)  # (input, hidden, num_layers)

        # score matrix
        self.W = nn.Linear(self.hiddenDim, clsNum)

    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        # CoM 모델의 출력
        batch_com_out = self.com_model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
        batch_com_final = batch_com_out[:, 0, :]  # [B, hidden_dim]

        # GRU 통과 --> PM 결과
        batch_pm_gru_final = []

        for PM_inputs in batch_PM_input:
            if PM_inputs:
                pm_outs = []
                for PM_input in PM_inputs:
                    pm_out = self.pm_model(PM_input)['last_hidden_state'][:, 0, :]  # [1, hidden_dim]
                    pm_outs.append(pm_out)
                pm_outs = torch.cat(pm_outs, 0).unsqueeze(1)  # [speaker_num, 1, hidden_dim]

                h0 = torch.zeros(2, pm_outs.size(1), self.hiddenDim).cuda()  # (num_layers, batch, hidden_size)
                pm_gru_outs, _ = self.speakerGRU(pm_outs, h0)  # [speaker_num, 1, hidden_dim]
                pm_gru_final = pm_gru_outs[-1, :, :]  # [1, hidden_dim]
                batch_pm_gru_final.append(pm_gru_final)
            else:
                batch_pm_gru_final.append(torch.zeros(1, self.hiddenDim).cuda())

        batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)  # [speaker_num, hidden_dim]

        # 최종 출력
        final_output = self.W(batch_com_final + batch_pm_gru_final)  # [B, C]

        return final_output


In [ ]:
!touch model.py

In [ ]:
!pip install torchmetrics

In [ ]:
import lightning.pytorch as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torchmetrics import F1Score

# from model import ERC_model

class ERCTrainer(pl.LightningModule):
    def __init__(self, clsNum, lr=5e-5, weight_decay=1e-2, warmup_step=32):
        super(ERCTrainer, self).__init__()
        self.model = ERC_model(clsNum)
        self.crit = nn.CrossEntropyLoss()
        self.lr = lr
        self.weight_decay = weight_decay
        self.warmup_steps = warmup_step

        self.f1_macro = F1Score(task='multiclass', num_classes=clsNum, average='macro')
        self.validation_outputs = []  # Validation 결과를 저장할 리스트

    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        return self.model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)

    # batch : collate fn return
    def training_step(self, batch, batch_idx):
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, labels = batch

        # self : forward
        logits = self(batch_padding_token, batch_padding_attention_mask, batch_PM_input)

        loss = self.crit(logits, labels)
        # 명시적으로 batch_size 전달
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=batch_padding_token.size(0))
        return loss

    def validation_step(self, batch, batch_idx):
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, labels = batch

        # self : forward
        logits = self(batch_padding_token, batch_padding_attention_mask, batch_PM_input)

        loss = self.crit(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()

        # F1 Score 업데이트 및 Validation 결과 저장
        self.f1_macro.update(preds, labels)
        self.validation_outputs.append(loss)

        # 명시적으로 batch_size 전달
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=batch_padding_token.size(0))
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=batch_padding_token.size(0))
        return loss

    def on_validation_epoch_end(self):
        # Validation epoch이 끝난 후 F1 Score 계산 및 로그
        macro_f1 = self.f1_macro.compute()
        self.log('val_macro_f1', macro_f1, prog_bar=True)

        # F1 Score 및 validation outputs 초기화
        self.f1_macro.reset()
        self.validation_outputs.clear()

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.warmup_steps, num_training_steps=self.trainer.estimated_stepping_batches
        )
        return [optimizer], [scheduler]


In [ ]:
!touch trainer.py

In [ ]:
!pip install --upgrade pip setuptools
!pip install -U mlflow

In [ ]:
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

# from dataloader import MeldDataLoader
import argparse
import sys
# from trainer import ERCTrainer

sys.argv = ['train.py', '--train_dir', '/content/MELD/data/MELD/train_sent_emo.csv',
            '--valid_dir', '/content/MELD/data/MELD/dev_sent_emo.csv', '--dataset_name', 'meld',
            '--gpu_id', '0', '--lr', '5e-5', '--lr_decay_start', '156']

def define_argparser():
  p = argparse.ArgumentParser()

  p.add_argument(
      '--train_dir',
      type=str,
      help='Set train file path'
  )
  p.add_argument(
      '--valid_dir',
      type=str,
      help='Set valid file path'
  )
  p.add_argument(
      '--dataset_name',
      type=str,
      default='meld',
      help='Set dataset name'
  )
  p.add_argument(
      '--batch_size',
      type=int,
      default=32,
      help='Mini batch size for gradient descent. Default=%(default)s'
  )
  p.add_argument(
      '--n_epochs',
      type=int,
      default=3,
      help='Number of epochs to train. Default=%(default)s'
  )
  p.add_argument(
      '--gpu_id',
      type=int,
      default=-1,
      help='GPU ID to train. Currently, GPU parallel is not supported. -1 for CPU. Default=%(defalut)s'
  )
  p.add_argument(
      '--context_window',
      type=int,
      default=3,
      help='Set context window size'
  )
  p.add_argument(
      '--lr',
      type=float,
      default=5e-5,
      help='Initial learning rate. Default=%(default)s',
  )
  p.add_argument(
      '--weight_decay',
      type=float,
      default=1e-2,
      help='Initial learning rate. Default=%(default)s',
  )
  p.add_argument(
      '--lr_decay_start',
      type=int,
      default=10,
      help='Learning rate decay start at. Default=%(default)s',
  )

  p.add_argument(
      '--resume_ckpt',
      type=str,
      default=None,
      help='resume train checkpoint path',
  )

  config = p.parse_args()

  return config

def main(config):
  mlflow_logger = pl_loggers.MLFlowLogger(save_dir='logs/')

  data_loader = MeldDataLoader(train_dir=config.train_dir, valid_dir=config.valid_dir,batch_size=config.batch_size,context_window=config.context_window)

  if config.dataset_name.lower() == 'meld':
    clsNum = 7

  checkpoint_callback = ModelCheckpoint(
    dirpath="/content/gdrive/MyDrive/model_checkpoints/",
    filename="{epoch}-{val_loss:.2f}",  # 각 체크포인트마다 고유한 이름 생성
    save_top_k=-1,  # 모든 체크포인트 저장
    monitor="val_loss",  # validation loss를 기준으로 체크포인트 선택
    mode="min",  # val_loss가 작을수록 더 좋은 모델로 간주
    every_n_epochs=1  # 매 에포크마다 체크포인트 저장
)

  model = ERCTrainer(clsNum=clsNum, lr=config.lr, weight_decay=config.weight_decay, warmup_step=config.lr_decay_start)
  trainer = Trainer(
    max_epochs=config.n_epochs,
    devices=[config.gpu_id] if config.gpu_id >= 0 else None,
    accelerator="gpu" if config.gpu_id >= 0 else "cpu",
    enable_progress_bar=True,
    logger=mlflow_logger,
    callbacks=[checkpoint_callback],
    check_val_every_n_epoch=1
  )

  if config.resume_ckpt is not None:
    trainer.fit(model, data_loader, ckpt_path=config.resume_ckpt)
  else:
    trainer.fit(model, data_loader)


if __name__ == '__main__':
  config = define_argparser()
  main(config)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [ ]:
# 코랩 안끊기게 하는법
# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End");
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();


# 중단하고 싶을 때
# stopClickConnect();

In [ ]:
!touch train.py

In [ ]:
import torch
from torch.utils.data import DataLoader

# 데이터셋 및 데이터로더 설정
test_dataset = MeldDataset('/content/MELD/data/MELD/test_sent_emo.csv')
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4, collate_fn=test_dataset.collate_fn)

clsNum = len(test_dataset.emoList)

# ERC_model을 클래스에서 불러오기
model_path = '/content/gdrive/MyDrive/model_checkpoints/epoch=2-val_loss=1.47.ckpt'
erc_model = ERCTrainer.load_from_checkpoint(model_path, clsNum=clsNum)  # load_from_checkpoint를 사용
erc_model.eval()
print('')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
def ErrorSamples(model, dataloader):
    model.eval()
    correct = 0
    label_list = []
    pred_list = []

    error_samples = []
    # label arragne
    with torch.no_grad():
        for i_batch, data in enumerate(tqdm(dataloader)):
            """Prediction"""
            batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
            batch_padding_token = batch_padding_token.cuda()
            batch_padding_attention_mask = batch_padding_attention_mask.cuda()
            batch_PM_input = [[x2.cuda() for x2 in x1] for x1 in batch_PM_input]
            batch_label = batch_label.cuda()

            """Prediction"""
            pred_logits = erc_model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)

            """Calculation"""
            pred_label = pred_logits.argmax(1).item()
            true_label = batch_label.item()

            pred_list.append(pred_label)
            label_list.append(true_label)
            if pred_label != true_label:
                error_samples.append([batch_padding_token, true_label, pred_label])
            if pred_label == true_label:
                correct += 1
        acc = correct/len(dataloader)
    return error_samples, acc, pred_list, label_list

In [ ]:
error_samples, acc, pred_list, label_list = ErrorSamples(erc_model, test_dataloader)

  0%|          | 0/2610 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 2610/2610 [02:18<00:00, 18.81it/s]


In [ ]:
print(f'accuracy : ', acc)

accuracy :  0.49386973180076627


In [ ]:
import random
random_error_samples = random.sample(error_samples, 5)

for random_error_sample in random_error_samples:
  batch_padding_token, true_label, pred_label = random_error_sample
  print('--------------------------------------------------------')
  print("입력 문장들: ", test_dataset.tokenizer.decode(batch_padding_token.squeeze(0).tolist()))
  print("정답 감정: ", test_dataset.emoList[true_label])
  print("예측 감정: ", test_dataset.emoList[pred_label])

--------------------------------------------------------
입력 문장들:  <s><s> Look what I got! Look what I got! Look what I got!  Can you believe they make these for little people?</s> Little village people.</s> Okay, look at this one. This is my favourite.</s></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [ ]:
import os
import shutil
from google.colab import drive

drive.mount('/content/drive')

source_path = '/content/logs/589356289968678222'

destination_path = '/content/drive/MyDrive/mlflow_logs/'

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
